In [ ]:
import fxcmpy
import time
import datetime as dt
from pyti.simple_moving_average import simple_moving_average as sma
from pyti.exponential_moving_average import exponential_moving_average as ema
import talib as ta
import numpy as np

### STRATEGY DESCRIPTION ####
# This strategy buys when the Fast SMA crosses over the Slow SMA and sell when the Fast SMA crosses
# under the Slow SMA. Opposing signals will close out opposing positions if close_on_opposing_signal == True.
# If close_on_opposing_signal == False, the strategy can open hedged positions (long & short at the same time)
# Parameters allow traders to change token, symbol, timeframe, Fast SMA & Slow SMA Periods, 
# Opposing signal closing logic, trade size/stop/limit.
# This is a close-of-bar strategy, meaning it only signals trades at the close of a bar.
# For more strategy examples, please visit github.com/fxcm/RestAPI
#############################

###### USER PARAMETERS ######
# token = '7363fec3b64647198252257812994375c9769f4d'
symbol = 'EUR/USD'
timeframe = "m1"	        # (m1,m5,m15,m30,H1,H2,H3,H4,H6,H8,D1,W1,M1)
fast_sma_periods = 10
slow_sma_periods = 20
close_on_opposing_signal = True
amount = 1
stop = -10
trailstop_value = 10
limit = 30
#############################

# Global Variables
pricedata = None
numberofcandles = 300


# === INPUTS ===
HiLoLen         = 34
fastEMAlength   = 89
mediumEMAlength = 200
slowEMAlength   = 600
Lookback   = 3
Delay           = 1

# === INPUTS ===

TradeDirection_arr = np.array([0.0,0.0])
close_arr = np.array([0.0,0.0])
pacC_arr = np.array([0.0,0.0])
        
        


# Connect to FXCM API
con = fxcmpy.fxcmpy(access_token="33f1a78a9a59a52df59dc8b61b2d6ba7f3b0e7d8", log_level="error")
# con = fxcmpy.fxcmpy(config_file='fxcm.cfg')
	
# This function runs once at the beginning of the strategy to run initial one-time processes/computations
def Prepare():
	global pricedata
	
	print("Requesting Initial Price Data...")
	pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	print(pricedata)
	print("Initial Price Data Received...")

# Get latest close bar prices and run Update() function every close of bar/candle
def StrategyHeartBeat():
	while True:
		currenttime = dt.datetime.now()
		if timeframe == "m1" and currenttime.second == 0 and GetLatestPriceData():
			Update()
		elif timeframe == "m5" and currenttime.second == 0 and currenttime.minute % 5 == 0 and GetLatestPriceData(): 
			Update()
			time.sleep(240)
		elif timeframe == "m15" and currenttime.second == 0 and currenttime.minute % 15 == 0 and GetLatestPriceData(): 
			Update()
			time.sleep(840)
		elif timeframe == "m30" and currenttime.second == 0 and currenttime.minute % 30 == 0 and GetLatestPriceData():
			Update()
			time.sleep(1740)
		elif currenttime.second == 0 and currenttime.minute == 0 and GetLatestPriceData():
			Update()
			time.sleep(3540)
		time.sleep(1)

# Returns True when pricedata is properly updated			
def GetLatestPriceData():
	global pricedata
	
	# Normal operation will update pricedata on first attempt
	new_pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	if new_pricedata.index.values[len(new_pricedata.index.values)-1] != pricedata.index.values[len(pricedata.index.values)-1]:
		pricedata= new_pricedata
		return True
		
	counter = 0
	# If data is not available on first attempt, try up to 3 times to update pricedata
	while new_pricedata.index.values[len(new_pricedata.index.values)-1] == pricedata.index.values[len(pricedata.index.values)-1] and counter < 3:
		print("No updated prices found, trying again in 10 seconds...")
		counter+=1
		time.sleep(10)
		new_pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	if new_pricedata.index.values[len(new_pricedata.index.values)-1] != pricedata.index.values[len(pricedata.index.values)-1]:
		pricedata = new_pricedata
		return True
	else:
		return False
		
# This function is run every time a candle closes
def Update():
	print(str(dt.datetime.now()) + "	 " + timeframe + " Bar Closed - Running Update Function...")
    

    
	closes = np.array(pricedata['bidclose'])
	lows = np.array(pricedata['bidlow'])
	highs = np.array(pricedata['bidhigh'])
	opens = np.array(pricedata['bidopen'])
    

    
	#Array Values
	fastEMA_arr = ta.EMA(closes, timeperiod=fastEMAlength) 
	mediumEMA_arr = ta.EMA(closes, timeperiod=mediumEMAlength) 
	slowEMA_arr = ta.EMA(closes, timeperiod=slowEMAlength) 
	pacC_arr = ta.EMA(closes, timeperiod=HiLoLen) 
	pacL_arr = ta.EMA(lows, timeperiod=HiLoLen) 
	pacU_arr = ta.EMA(highs, timeperiod=HiLoLen) 

    #Single Values
	fastEMA = fastEMA_arr[len(fastEMA_arr)-1]
	mediumEMA = mediumEMA_arr[len(mediumEMA_arr)-1]
	slowEMA = slowEMA_arr[len(slowEMA_arr)-1]
	pacC = pacC_arr[len(pacC_arr)-1]
	pacL = pacL_arr[len(pacL_arr)-1]
	pacU = pacU_arr[len(pacU_arr)-1]
    
    
    
    
    
    
    ## You can access the TradeBar dictionary in the slice object and then subset by symbol
    ## to get the TradeBar for IBM
	open_ = opens[len(opens)-1]      ## Open price
	close = closes[len(closes)-1]    ## Close price
	high = highs[len(highs)-1]    ## High price
	low = lows[len(lows)-1]    ## Low price
        
	pacExitU = False
	pacExitL = False
        
	Buy = False
	Sell = False
        
	Long = False
	Short = False
	Close_trade = False

    

    # === SERIES SETUP ===    
	TrendDirection = 0
        
	if fastEMA > mediumEMA and pacL > mediumEMA: 
		TrendDirection = 1
	elif fastEMA < mediumEMA and pacU < mediumEMA:
		TrendDirection = -1 
	else: 
		TrendDirection = 0    

    
    
    # === ALERTING ===
    # Initialise Trading state.
	global TradeDirection_arr
	global close_arr
	global pacC_arr
    
	if TradeDirection_arr[0] is None:
		TradeDirection = 0
	else:
		TradeDirection = TradeDirection_arr[0]    

        
        
    # barssince(close<pacC)
	i = 0
	while close_arr[i] > pacC_arr[i]:
		i += 1
          
    # barssince(close>pacC) 
	j = 0
	while close_arr[j] < pacC_arr[j]:
		j += 1
          
        
        
	if open_ < pacU and close > pacU and i <= Lookback:
		pacExitU = True
	elif open_ > pacL and close < pacL and j <= Lookback:
		pacExitL = True
        
        
	if TrendDirection == 1 and pacExitU:
		Buy = True
	elif TrendDirection == -1 and pacExitL:
		Sell = True
        
            
        
    # Keep Current trading state until Pullback occurs or New Recovery.
	if TradeDirection == 1 and close < pacC:
		TradeDirection = 0
	elif TradeDirection == -1 and close > pacC:
		TradeDirection = 0
	elif TradeDirection == 0 and Buy:
		TradeDirection = 1
	elif TradeDirection == 0 and Sell:
		TradeDirection = -1
	else:
		TradeDirection = TradeDirection
            
            
	if TradeDirection_arr[0] == 0 and TradeDirection == 1:
		Long = True
	elif TradeDirection_arr[0] == 0 and TradeDirection == -1:
		Short = True
	elif TradeDirection_arr[0] == 1 and TradeDirection == 0:
		Close_trade = True
	elif TradeDirection_arr[0] == -1 and TradeDirection == 0:
		Close_trade = True
        
        
	TradeDirection_arr = np.insert(TradeDirection_arr, 0, TradeDirection)  
    
    
  
    
        
    #TRADING LOGIC
    #Buy If colFinal is aqua and colFinal2 is lime
	if Long == True :
		print("	  BUY SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("S") > 0:
			print("	  Closing Sell Trade(s)...")
			exit("S")
		if countOpenTrades("B") == 0:
			print("	  Opening Buy Trade...")
			enter("B")
        
    #Sell If colFinal is orange and colFinal2 is red
	if Short == True :
		print("	  SELL SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("B") > 0:
			print("	  Closing Buy Trade(s)...")
			exit("B")
		if countOpenTrades("S") == 0:
			print("	  Opening Sell Trade...")
			enter("S")
        
    #EXIT STRTEGY
	if Close_trade :
		print("	  EXIT SELL SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("S") > 0:
			print("	  Closing Sell Trade(s)...")
			exit("S")
		print("	  EXIT BUY SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("B") > 0:
			print("	  Closing Buy Trade(s)...")
			exit("B")
            
    
	
		
	print(str(dt.datetime.now()) + "	 " + timeframe + " Update Function Completed.\n")

# This function places a market order in the direction BuySell, "B" = Buy, "S" = Sell, uses symbol, amount, stop, limit
def enter(BuySell):
	direction = True;
	if BuySell == "S":
		direction = False;
	try:
		opentrade = con.open_trade(symbol=symbol, is_buy=direction,amount=amount, time_in_force='GTC',order_type='AtMarket',is_in_pips=True,limit=limit, stop=stop, trailing_step=trailstop_value)
	except:
		print("	  Error Opening Trade.")
	else:
		print("	  Trade Opened Successfully.")

# This function closes all positions that are in the direction BuySell, "B" = Close All Buy Positions, "S" = Close All Sell Positions, uses symbol
def exit(BuySell=None):
	openpositions = con.get_open_positions(kind='list')
	isbuy = True
	if BuySell == "S":
		isbuy = False
	for position in openpositions:
		if position['currency'] == symbol:
			if BuySell is None or position['isBuy'] == isbuy:
				print("	  Closing tradeID: " + position['tradeId'])
				try:
					closetrade = con.close_trade(trade_id=position['tradeId'], amount=position['amountK'])
				except:
					print("	  Error Closing Trade.")
				else:
					print("	  Trade Closed Successfully.")

# Returns number of Open Positions for symbol in the direction BuySell, returns total number of both Buy and Sell positions if no direction is specified
def countOpenTrades(BuySell=None):		
	openpositions = con.get_open_positions(kind='list')
	isbuy = True
	counter = 0
	if BuySell == "S":
		isbuy = False
	for position in openpositions:
		if position['currency'] == symbol:
			if BuySell is None or position['isBuy'] == isbuy:
				counter+=1
	return counter

Prepare() # Initialize strategy
StrategyHeartBeat() # Run strategy

<ipython-input-1-a9f7618d23b7>:179: SyntaxWarning: name 'pacC_arr' is assigned to before global declaration
  global pacC_arr


Requesting Initial Price Data...
                     bidopen  bidclose  bidhigh   bidlow  askopen  askclose  \
date                                                                          
2020-03-30 11:08:00  1.10677   1.10669  1.10678  1.10662  1.10691   1.10684   
2020-03-30 11:09:00  1.10669   1.10672  1.10695  1.10668  1.10684   1.10686   
2020-03-30 11:10:00  1.10672   1.10677  1.10686  1.10665  1.10686   1.10691   
2020-03-30 11:11:00  1.10677   1.10616  1.10677  1.10616  1.10691   1.10630   
2020-03-30 11:12:00  1.10616   1.10663  1.10674  1.10614  1.10630   1.10675   
2020-03-30 11:13:00  1.10663   1.10630  1.10668  1.10627  1.10675   1.10645   
2020-03-30 11:14:00  1.10630   1.10617  1.10639  1.10617  1.10645   1.10631   
2020-03-30 11:15:00  1.10617   1.10607  1.10626  1.10595  1.10631   1.10620   
2020-03-30 11:16:00  1.10607   1.10581  1.10618  1.10580  1.10620   1.10594   
2020-03-30 11:17:00  1.10581   1.10556  1.10586  1.10554  1.10594   1.10568   
2020-03-30 11:18:00

No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
2020-03-30 16:08:34.403833	 m1 Bar Closed - Running Update Function...
2020-03-30 16:08:34.403833	 m1 Update Function Completed.

No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
2020-03-30 16:09:33.823740	 m1 Bar Closed - Running Update Function...
2020-03-30 16:09:33.823740	 m1 Update Function Completed.

No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
2020-03-30 16:10:34.284796	 m1 Bar Closed - Running Update Function...
2020-03-30 16:10:34.285752	 m1 Update Function Completed.

No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
2020-

No updated prices found, trying again in 10 seconds...
2020-03-30 16:35:34.394590	 m1 Bar Closed - Running Update Function...
2020-03-30 16:35:34.394590	 m1 Update Function Completed.

No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
2020-03-30 16:36:33.749854	 m1 Bar Closed - Running Update Function...
2020-03-30 16:36:33.749854	 m1 Update Function Completed.

No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
2020-03-30 16:37:34.198276	 m1 Bar Closed - Running Update Function...
2020-03-30 16:37:34.199273	 m1 Update Function Completed.

No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
No updated prices found, trying again in 10 seconds...
2020-03-30 16:38:33.949520	 m1 Bar Closed - Running Update F